# Global Daily Event Analysis: Marine Heatwave ID & Tracking using `MarEx`

### `MarEx` Processing Pipeline for Gridded Datasets:

1. **Morphological Pre-Processing**
    - Performs binary morphological closing using `dask_image.ndmorph` to fill small spatial holes up to `R_fill` cells in radius 
    - Executes binary opening to remove isolated small features of order `R_fill`
    - Fills gaps in time to maintain event continuity for interruptions up to `T_fill` time steps
    - Filters out smallest objects below the `area_filter_quartile` percentile threshold

2. **Blob Identification**
    - Labels spatially connected components using efficient connected-component algorithm in `dask_image.ndmeasure`
    - Computes blob properties (area, centroid, boundaries)

3. **Temporal Tracking**
    - Identifies blob overlaps between consecutive time frames
    - Connects objects across time, applying the following criteria for splitting, merging, & persistence:
        - Connected objects must overlap by at least fraction `overlap_threshold` of the smaller area
        - Merged objects retain their original ID, but partition the child area based on the parent of the _nearest-neighbour_ cell (or centroid distance)

4. **Graph Reduction & Finalisation**
    - Constructs the complete temporal graph of object evolution through time
    - Resolves object connectivity graph using `scipy.sparse.csgraph.connected_components`
    - Creates globally unique IDs for each tracked extreme event
    - Maps objects into efficient ID-time space for convenient analysis
    - Computes comprehensive statistics about the lifecycle of each event

The pipeline leverages **dask** for distributed parallel computation, enabling efficient processing of large datasets. \
A 40-year global daily analysis at 0.25° resolution on 32 cores takes
- Basic (i.e. Scannell et al., which involves no merge/split criteria or tracking):  ~5 minutes
- Full Split/Merge Thresholding & Merge Tracking:  ~40 minutes

In [1]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [ ]:
# Lustre Scratch Directory
scratch_dir = Path('/scratch') / getuser()[0] / getuser()

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=32, threads_per_worker=1,
                                 scratch_dir = scratch_dir / 'clients')  # Specify temporary scratch directory for dask to use

Dask Scratch: '/scratch/b/b382615/clients/tmpsmzaypgs'
Memory per Worker: 15.74 GB


Hostname: l40095
Forward Port: l40095:38675
Dashboard Link: localhost:38675/status


In [ ]:
# Choose optimal chunk size & load data
#   N.B.: This is crucial for dask (not only for performance, but also to make the problem tractable)
#         The operations are eventually global-in-space, and so requires the spatial dimension to be contiguous/unchunked
#         We can adjust the chunk size in time depending on available system memory.

chunk_size = {'time': 25, 'lat': -1, 'lon': -1}

In [5]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = scratch_dir / 'mhws' / 'extremes_binary_gridded.zarr'
ds = xr.open_zarr(str(file_name), chunks=chunk_size)

In [6]:
# Run ID, Tracking, & Merging

tracker = marEx.tracker(ds.extreme_events, 
                       ds.mask.where((ds.lat < 85) &
                                     (ds.lat > -90), other=False),  # Modify Mask: Anisotropy of the lat/lon grid near the poles biases the ID & Tracking
                       area_filter_quartile = 0.5,                  # Remove the smallest 50% of the identified coherent extreme areas
                       R_fill = 8,                                  # Fill small holes with radius < 8 _cells_
                       T_fill = 2,                                  # Allow gaps of 2 days and still continue the event tracking with the same ID
                       allow_merging = True,                        # Allow extreme events to split/merge. Keeps track of merge events & unique IDs.
                       overlap_threshold = 0.5,                     # Overlap threshold for merging events. If overlap < threshold, events keep independent IDs.
                       nn_partitioning = True,                      # Use new NN method to partition merged children areas. If False, reverts to old method of Di Sun et al. 2023.
                       verbosity=1)                                 # Choose Verbosity Level (0=None, 1=Basic, 2=Advanced/Timing)

extreme_events_ds, merges_ds = tracker.run(return_merges=True)
extreme_events_ds

Finished filling spatial holes
Finished filling spatio-temporal holes
Finished filtering small objects
Finished object identification
Finished calculating object properties
Finished finding overlapping objects


Processing splitting and merging in chunk 0 of 322
Processing splitting and merging in chunk 10 of 322
...
Processing splitting and merging in chunk 320 of 322
Finished splitting and merging objects
Finished clustering and renaming objects into coherent consistent events
Finished tracking all extreme events!


Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.8129384517472887
   Total Object Area IDed (cells): 379171404.0
   Number of Initial Pre-Filtered Objects: 206336
   Number of Final Filtered Objects: 103227
   Area Cutoff Threshold (cells): 694
   Accepted Area Fraction: 0.906774014002385
   Total Events Tracked: 9704
   Total Merging Events Recorded: 19243


<xarray.Dataset> Size: 40GB
Dimensions:       (lat: 721, lon: 1440, time: 8036, ID: 9705, component: 2,
                   sibling_ID: 14)
Coordinates:
  * lat           (lat) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * lon           (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time          (time) datetime64[ns] 64kB 2020-01-01 ... 2041-12-31
  * ID            (ID) int32 39kB 1 2 3 4 5 6 ... 9700 9701 9702 9703 9704 9705
Dimensions without coordinates: component, sibling_ID
Data variables:
    ID_field      (time, lat, lon) int32 33GB dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
    global_ID     (time, ID) int32 312MB dask.array<chunksize=(25, 9705), meta=np.ndarray>
    area          (time, ID) float32 312MB dask.array<chunksize=(25, 9705), meta=np.ndarray>
    centroid      (component, time, ID) float64 1GB dask.array<chunksize=(2, 25, 9705), meta=np.ndarray>
    presence      (time, ID) bool 78MB dask.array<chunksize=(25, 9705), meta=np.ndarray>
    time_start    (ID) datetime64[ns] 78kB dask.array<chunksize=(9705,), meta=np.ndarray>
    time_end      (ID) datetime64[ns] 78kB dask.array<chunksize=(9705,), meta=np.ndarray>
    merge_ledger  (time, ID, sibling_ID) int32 4GB dask.array<chunksize=(25, 9705, 14), meta=np.ndarray>
Attributes: (12/14)
    allow_merging:               1
    N_objects_prefiltered:       206336
    N_objects_filtered:          103227
    N_events_final:              9704
    R_fill:                      8
    T_fill:                      2
    ...                          ...
    accepted_area_fraction:      0.906774014002385
    preprocessed_area_fraction:  0.8129384517472887
    overlap_threshold:           0.5
    nn_partitioning:             1
    total_merges:                19243
    multi_parent_merges:         4449

In [7]:
merges_ds

<xarray.Dataset> Size: 3MB
Dimensions:        (merge_ID: 19243, parent_idx: 14, child_idx: 14)
Dimensions without coordinates: merge_ID, parent_idx, child_idx
Data variables:
    parent_IDs     (merge_ID, parent_idx) int32 1MB 15 16 -1 -1 ... -1 -1 -1 -1
    child_IDs      (merge_ID, child_idx) int32 1MB 29 103228 -1 -1 ... -1 -1 -1
    overlap_areas  (merge_ID, parent_idx) int32 1MB 1437 7652 -1 -1 ... -1 -1 -1
    merge_time     (merge_ID) datetime64[ns] 154kB 2020-01-03 ... 2041-12-29
    n_parents      (merge_ID) int8 19kB 2 2 2 2 2 2 2 3 3 ... 3 1 3 3 3 2 2 2 2
    n_children     (merge_ID) int8 19kB 2 2 2 2 2 2 2 3 3 ... 3 1 3 3 3 2 2 2 2
Attributes:
    fill_value:  -1

In [8]:
# Save IDed/Tracked/Merged Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / 'mhws' / 'extreme_events_merged_gridded.zarr'
extreme_events_ds.to_zarr(file_name, mode='w')

### Run Basic Tracking for Comparison
N.B.: This is the current standard method used in the literature, which involves _No_ temporal gap filling, _No_ merging/splitting and _No_ independent event tracking.

In [9]:
# Run Basic Tracking

tracker = marEx.tracker(ds.extreme_events, 
                       ds.mask.where((ds.lat < 85) &
                                     (ds.lat > -90), other=False),  # Modify Mask: Anisotropy of the lat/lon grid near the poles biases the ID & Tracking
                       area_filter_quartile = 0.5,                  # Remove the smallest 50% of the identified coherent extreme areas
                       R_fill = 8,                                  # Fill small holes with radius < 8 _cells_
                       T_fill = 0,                                  # No temporal hole filling
                       allow_merging = False,                       # Do not allow extreme events to split/merge. All touching events adopt the same ID forever (after _and_ before (!)).
                       verbosity=1)

extreme_events_basic_ds = tracker.run()
extreme_events_basic_ds

Finished filling spatial holes
Finished filling spatio-temporal holes
Finished filtering small objects
Finished tracking all extreme events!


Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.8525119484330325
   Total Object Area IDed (cells): 361690641.0
   Number of Initial Pre-Filtered Objects: 210966
   Number of Final Filtered Objects: 105545
   Area Cutoff Threshold (cells): 651
   Accepted Area Fraction: 0.9064810029187347
   Total Events Tracked: 10190




<xarray.Dataset> Size: 33GB
Dimensions:   (lat: 721, lon: 1440, time: 8036)
Coordinates:
  * lat       (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon       (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time      (time) datetime64[ns] 64kB 2020-01-01 2020-01-02 ... 2041-12-31
Data variables:
    ID_field  (time, lat, lon) int32 33GB dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
Attributes:
    allow_merging:               0
    N_objects_prefiltered:       210966
    N_objects_filtered:          105545
    N_events_final:              10190
    R_fill:                      8
    T_fill:                      0
    area_filter_quartile:        0.5
    area_threshold (cells):      651.0
    accepted_area_fraction:      0.9064810029187347
    preprocessed_area_fraction:  0.8525119484330325

In [10]:
# Save IDed Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / 'mhws' / 'extreme_events_basic_gridded.zarr'
extreme_events_basic_ds.to_zarr(file_name, mode='w')